# Data Cleaning/Wrangling for Springboard Capstone Project 1: 
## Prediction of hospital readmission rate for patients with an existing diagnosis based on factors measured at time of intial admission.
The dataset I will be using was originally constructed by researchers at Virginia Commonwealth University to see whether the decision to take a measurement of HbA1c (a test to measure glucose in diabetic patients) during hospitalization led to lower rate of hospital readmission. The HbA1c test is considered by the researchers to be a proxy for a more active management of the diabetes in the patient.  

In this analysis, I will repurpose the data set to perform a correlational analysis/ predictive modeling to determine whether one or more variables are predictive of whether or not a patient will be readmitted to the hospital within 30 days of discharge. Time allowing, the analysis may be extended to see whether prediction of readmission after 30 days can also be predicted.

In [1]:
import pandas as pd
import numpy as np

The data set, ‘Diabetes 130-US hospitals for years 1999-2008 Data Set’ is an open dataset that is obtainable at the UCI Machine Learning repository:
https://archive.ics.uci.edu/ml/datasets/Diabetes+130-US+hospitals+for+years+1999-2008# 

In [2]:
file = 'diabetic_data.csv'
df = pd.read_csv(file)

In [3]:
# df.sample with number of rows in parenthesis returns a random sample of values, which may be more informative than the first
# 5 rows as would be returned from df.head()
df.sample(5)

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
82258,256141560,68382819,Caucasian,Male,[70-80),?,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
60607,169637514,40829382,Caucasian,Female,[50-60),?,1,1,7,1,...,No,No,No,No,No,No,No,No,No,NO
37330,115624470,89373519,Caucasian,Female,[50-60),?,2,3,7,7,...,No,No,No,No,No,No,No,No,Yes,NO
54647,158775006,63704169,AfricanAmerican,Female,[70-80),?,1,3,7,8,...,No,Steady,No,No,No,No,No,No,Yes,NO
9489,41351562,13291434,AfricanAmerican,Male,[60-70),?,2,1,4,10,...,No,Steady,No,No,No,No,No,Ch,Yes,NO


In [4]:
# Use df.info() find out the basic parameters of the data.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101766 entries, 0 to 101765
Data columns (total 50 columns):
encounter_id                101766 non-null int64
patient_nbr                 101766 non-null int64
race                        101766 non-null object
gender                      101766 non-null object
age                         101766 non-null object
weight                      101766 non-null object
admission_type_id           101766 non-null int64
discharge_disposition_id    101766 non-null int64
admission_source_id         101766 non-null int64
time_in_hospital            101766 non-null int64
payer_code                  101766 non-null object
medical_specialty           101766 non-null object
num_lab_procedures          101766 non-null int64
num_procedures              101766 non-null int64
num_medications             101766 non-null int64
number_outpatient           101766 non-null int64
number_emergency            101766 non-null int64
number_inpatient            10176

In addition to finding out that there are 50 variables and 101766 entries, we also find out that none of the columns have non-null values. This could either mean there are no missing values (unlikely) or that the missing values are denoted as either blank or some string (e.g., '?') or numeric (e.g., 0, -1) character.

In [5]:
colnames = df.columns.tolist()

In [6]:
# Show the unique values for each column as first step to identifying how missing values are shown, 
# issues with binary columns, etc.
for name in colnames:
    print(name, df[name].unique())

encounter_id [  2278392    149190     64410 ... 443854148 443857166 443867222]
patient_nbr [  8222157  55629189  86047875 ... 140199494 120975314 175429310]
race ['Caucasian' 'AfricanAmerican' '?' 'Other' 'Asian' 'Hispanic']
gender ['Female' 'Male' 'Unknown/Invalid']
age ['[0-10)' '[10-20)' '[20-30)' '[30-40)' '[40-50)' '[50-60)' '[60-70)'
 '[70-80)' '[80-90)' '[90-100)']
weight ['?' '[75-100)' '[50-75)' '[0-25)' '[100-125)' '[25-50)' '[125-150)'
 '[175-200)' '[150-175)' '>200']
admission_type_id [6 1 2 3 4 5 8 7]
discharge_disposition_id [25  1  3  6  2  5 11  7 10  4 14 18  8 13 12 16 17 22 23  9 20 15 24 28
 19 27]
admission_source_id [ 1  7  2  4  5  6 20  3 17  8  9 14 10 22 11 25 13]
time_in_hospital [ 1  3  2  4  5 13 12  9  7 10  6 11  8 14]
payer_code ['?' 'MC' 'MD' 'HM' 'UN' 'BC' 'SP' 'CP' 'SI' 'DM' 'CM' 'CH' 'PO' 'WC' 'OT'
 'OG' 'MP' 'FR']
medical_specialty ['Pediatrics-Endocrinology' '?' 'InternalMedicine'
 'Family/GeneralPractice' 'Cardiology' 'Surgery-General' 'Orthopedic

metformin-rosiglitazone ['No' 'Steady']
metformin-pioglitazone ['No' 'Steady']
change ['No' 'Ch']
diabetesMed ['No' 'Yes']
readmitted ['NO' '>30' '<30']


In [7]:
# Function to calculate missing values
def per_miss_val(name):
    if name in ['race', 'weight', 'payer_code', 'medical_specialty', 'diag_1', 'diag_2', 'diag_3']:
        return (100*df.loc[:, name][df.loc[:,name]=='?'].count()/df.loc[:,name].count())
    elif name == 'gender':
        return (100*df.loc[:, name][df.loc[:,name]=='Unknown/Invalid'].count()/df.loc[:,name].count())
    else: 
        return 0.0

In [8]:
# Create a list of dictionaries where each column is a dictionary, the key is the column name, the value is a list of values.
records = []
for name in colnames:
    record = {}
  #  print(name) - used during debugging
    record['Feature Name'] = name
#    record['List of Unique Values'] = df[name].unique()
    record['# of Unique Values'] = len(df[name].unique())
    record['% Missing Values'] = per_miss_val(name)
    record['First Value'] = df[name].iloc[0]
    records.append(record)
    

In [9]:
feature_df = pd.DataFrame(records)
feature_df




,# of Unique Values,% Missing Values,Feature Name,First Value
0,101766,0.000000,encounter_id,2278392
1,71518,0.000000,patient_nbr,8222157
2,6,2.233555,race,Caucasian
3,3,0.002948,gender,Female
4,10,0.000000,age,[0-10)
5,10,96.858479,weight,?
6,8,0.000000,admission_type_id,6
7,26,0.000000,discharge_disposition_id,25
8,17,0.000000,admission_source_id,1
9,14,0.000000,time_in_hospital,1


# Missing Values

By inspection of unique values in each column:
1. race, weight, payer_code, medical specialty, diag_1, diag_2, diag_3 all use '?' to denote a missing value. 
2. gender uses 'Unknown/Invalid'
3. encounter_id and patient_nbr are shown as an integer type which means there are no strings.  Missing value could be shown as an integer (check if 0 or 1 in these columns).

Plan for missing values: Check how many missing values are in the column. If too many missing values, consider dropping the column. If reasonable amount, replace the missing values with NaN (not a number, null type). Decide on a strategy for filling the missing value and implement.


In [11]:
# Replace '?' with NaN for the race column
df.race.replace('?', np.nan, inplace= True)

In [31]:
# Replace 'Unknown/Invalid' with NaN for the gender column
df.gender.replace('Unknown/Invalid', np.nan, inplace=True)

The weight variable is mostly unknown - 97% missing.  I have no reason at this point to believe that it would be crucial to the model so  I've decided to not use this variable in the analysis and remove the column. 

An explanation of why so many values are missing in this column in found in the article describing the original work: “Large percentage of missing values of the weight attribute can be explained by the fact that prior to the HITECH legislation of the American Reinvestment and Recovery Act in 2009 hospitals and clinics were not required to capture it in a structured format.” (https://www.hindawi.com/journals/bmri/2014/781670/)


In [34]:
# Drop the weight column, rename dataframe to df1
df1 = df.drop('weight',axis=1)

In [36]:
# Replace the '?' with NaN for the payer code column
df1.payer_code.replace('?', np.nan, inplace = True)

In [38]:
# Replace the '?' with NaN for the medical specialty column
df1.medical_specialty.replace('?', np.nan, inplace = True)

In [40]:
# Replace the '?' with NaN for diag_1
df1.diag_1.replace('?', np.nan, inplace = True)

In [42]:
# Replace the '?' with NaN for diag_2
df1.diag_2.replace('?', np.nan, inplace = True)

In [44]:
# Replace the '?' with NaN for diag_3
df1.diag_3.replace('?', np.nan, inplace = True)

In [45]:
# Check for missing values denoted by a 0 or 1 or other unusual number in encounter_id and patient_nbg
df1.loc[:,['encounter_id', 'patient_nbr']].min()

encounter_id    12522
patient_nbr       135
dtype: int64

Encounter id and patient number appear to not have missing values.

In [52]:
# Check that replacements were done correctly
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101766 entries, 0 to 101765
Data columns (total 49 columns):
encounter_id                101766 non-null int64
patient_nbr                 101766 non-null int64
race                        99493 non-null object
gender                      101763 non-null object
age                         101766 non-null object
admission_type_id           101766 non-null int64
discharge_disposition_id    101766 non-null int64
admission_source_id         101766 non-null int64
time_in_hospital            101766 non-null int64
payer_code                  61510 non-null object
medical_specialty           51817 non-null object
num_lab_procedures          101766 non-null int64
num_procedures              101766 non-null int64
num_medications             101766 non-null int64
number_outpatient           101766 non-null int64
number_emergency            101766 non-null int64
number_inpatient            101766 non-null int64
diag_1                      101745 no

## The payer_code and medical_specialties columns each have almost half of their values missing
However, these might be very useful variables in terms of predictive value so I'm reluctant to drop the column.  Will revisit during the EDA analysis.
Perhaps the medical specialty code be used to generate a payer code?


In [53]:
# What percentage of payer_code and medical_specialty is missing?
100*(df1.loc[:,['payer_code', 'medical_specialty']].isnull().sum()/(df1.loc[:,['payer_code', 'medical_specialty']].count() + df1.loc[:,['payer_code', 'medical_specialty']].isnull().sum()))

payer_code           39.557416
medical_specialty    49.082208
dtype: float64

## Target Column preparation
The target column is 'readmitted'. Initially I will transform the column to binary (0 for 'No' and '>30', 1 for '<30').

Less than 30 days ('<30') is the standard for assessing what constitutes ‘hospital readmission’: CMS defines a hospital readmission as "an admission to an acute care hospital within 30 days of discharge from the same or another acute care hospital."Readmissions-Reduction-Program". www.cms.gov. 2016-02-04. Retrieved 2016-03-01.Also: https://www.cms.gov/Medicare/Medicare-Fee-for-Service-Payment/PhysicianFeedbackProgram/Downloads/2015-ACR-MIF.pdf

This is the criteria that makes sense for the initial mock business case discussed in the proposal.

However, I might want to extend the analysis for practice. A Kaggle competition which used the same data set (a closed competition, so no kernels posted) suggested that the expected output should include '>30' as a separate category (see https://www.kaggle.com/c/diabetes-hospital-readmission#evaluation). I will create a second column to include this as a separate category then decide whether to do multiclass analysis later on if time allows.


In [54]:
df1.readmitted.head(15)

0      NO
1     >30
2      NO
3      NO
4      NO
5     >30
6      NO
7     >30
8      NO
9      NO
10    >30
11    <30
12    <30
13     NO
14    >30
Name: readmitted, dtype: object

In [55]:
# Define a function to convert readmitted column to binary
def recode_readmit_bin(readmit_value):
    # Return 1 if readmit_value is '<30'
    if readmit_value == '<30':
        return 1
    # Return 0 if readmit_value is 'NO' or '>30'
    elif (readmit_value == 'NO')  or (readmit_value == '>30'):
        return 0
    # Return np.nan if anything else
    else:
        return np.nan
# Apply the function to the readmitted column and create new column
df1['readmit_bin'] = df1.readmitted.apply(recode_readmit_bin)
print(df1.loc[:15,['readmitted','readmit_bin']])

   readmitted  readmit_bin
0          NO            0
1         >30            0
2          NO            0
3          NO            0
4          NO            0
5         >30            0
6          NO            0
7         >30            0
8          NO            0
9          NO            0
10        >30            0
11        <30            1
12        <30            1
13         NO            0
14        >30            0
15         NO            0


In [56]:
# Check for missing values
df1.readmit_bin.isnull().sum()

0

In [57]:
# Define a function to convert readmitted column to three trinary
def recode_readmit_tri(readmit_value):
    # Return 1 if readmit_value is '<30'
    if readmit_value == '<30':
        return 1
    # Return 2 if readmit_value is '>30'
    elif readmit_value == '>30':
        return 2
    # Return 0 if readmit_value is 'NO'
    elif readmit_value == 'NO':
        return 0
    # Return np.nan if anything else
    else: 
        return np.nan

# Aplly the function to the readmitted column and create new column
df1['readmit_tri'] = df1.readmitted.apply(recode_readmit_tri)
print(df1.loc[:15,['readmitted','readmit_bin', 'readmit_tri']])



   readmitted  readmit_bin  readmit_tri
0          NO            0            0
1         >30            0            2
2          NO            0            0
3          NO            0            0
4          NO            0            0
5         >30            0            2
6          NO            0            0
7         >30            0            2
8          NO            0            0
9          NO            0            0
10        >30            0            2
11        <30            1            1
12        <30            1            1
13         NO            0            0
14        >30            0            2
15         NO            0            0


## Plan for handling rows with NaN in various columns
There are now null values in the following columns: race, payer_code, medical specialty, diag_1, diag_2, diag_3 and gender.

Race: 2.2%, diag_1: 0.02 %, diag_2: 0.35%, diag_3: 1.4%, gender: 0.002%, payer_code: 39.6%, medical_specialty:49.1

At this point, we don't know which variables are going to be the most important.  Not sure at what point dropping rows makes sense.  Also, there may be a way to fill in payer code or medical specialty from diagnosis?


In [12]:
# To understand each variable better, create a table showing the names of the columns, data type, 
# first row's values, and descriptions from the data dictionary

data_dictionary = pd.read_csv('diabetes-data-dict2.csv')
data_dictionary

,Feature name,Type,Description and values,% missing
0,Encounter ID,Numeric,Unique identifier of an encounter,0%
1,Patient number,Numeric,Unique identifier of a patient,0%
2,Race,Nominal,"Values: Caucasian, Asian, African American, Hi...",2%
3,Gender,Nominal,"Values: male, female, and unknown/invalid",0%
4,Age,Nominal,"Grouped in 10-year intervals: 0, 10), 10, 20),...",0%
5,Weight,Numeric,Weight in pounds.,97%
6,Admission type,Nominal,Integer identifier corresponding to 9 distinct...,0%
7,Discharge disposition,Nominal,Integer identifier corresponding to 29 distinc...,0%
8,Admission source,Nominal,Integer identifier corresponding to 21 distinc...,0%
9,Time in hospital,Numeric,Integer number of days between admission and d...,0%


In [61]:
# expand row 26 so each medication is on a separate row
drugs = data_dictionary.iloc[26][2]
drugs

'For the generic names: metformin, repaglinide, nateglinide, chlorpropamide, glimepiride, acetohexamide, glipizide, glyburide, tolbutamide, pioglitazone, rosiglitazone, acarbose, miglitol, troglitazone, tolazamide, examide, sitagliptin, insulin, glyburide-metformin, glipizide-metformin, glimepiride-pioglitazone, metformin-rosiglitazone, and metformin-pioglitazone, the feature indicates whether the drug was prescribed or there was a change in the dosage. Values: â€œupâ€\x9d if the dosage was increased during the encounter, â€œdownâ€\x9d if the dosage was decreased, â€œsteadyâ€\x9d if the dosage did not change, and â€œnoâ€\x9d if the drug was not prescribed'

In [62]:
# import regex library
import re

In [63]:
drug_list = re.compile('For the generic names: (.+?), the').findall(drugs)

In [64]:
drug_list2 = drug_list[0].split(', ')

In [65]:
drug_list2

['metformin',
 'repaglinide',
 'nateglinide',
 'chlorpropamide',
 'glimepiride',
 'acetohexamide',
 'glipizide',
 'glyburide',
 'tolbutamide',
 'pioglitazone',
 'rosiglitazone',
 'acarbose',
 'miglitol',
 'troglitazone',
 'tolazamide',
 'examide',
 'sitagliptin',
 'insulin',
 'glyburide-metformin',
 'glipizide-metformin',
 'glimepiride-pioglitazone',
 'metformin-rosiglitazone',
 'and metformin-pioglitazone']

In [66]:
drug_list2[-1] = 'metformin-pioglitazone'

In [67]:
#Create a dataframe with the drug names

df_drug = pd.DataFrame(drug_list2, columns = ["Feature name"])
df_drug.head()

,Feature name
0,metformin
1,repaglinide
2,nateglinide
3,chlorpropamide
4,glimepiride


In [69]:
drugs_desc = "the feature indicates whether the drug was prescribed or there was a change in the dosage. Values: “up” if the dosage was increased during the encounter, “down” if the dosage was decreased, “steady” if the dosage did not change, and “no” if the drug was not prescribed"

In [68]:
pd.concat([data_dictionary.iloc[:26,:],df_drug, data_dictionary.iloc[[27]]], ignore_index=True)

,% missing,Description and values,Feature name,Type
0,0%,Unique identifier of an encounter,Encounter ID,Numeric
1,0%,Unique identifier of a patient,Patient number,Numeric
2,2%,"Values: Caucasian, Asian, African American, Hi...",Race,Nominal
3,0%,"Values: male, female, and unknown/invalid",Gender,Nominal
4,0%,"Grouped in 10-year intervals: 0, 10), 10, 20),...",Age,Nominal
5,97%,Weight in pounds.,Weight,Numeric
6,0%,Integer identifier corresponding to 9 distinct...,Admission type,Nominal
7,0%,Integer identifier corresponding to 29 distinc...,Discharge disposition,Nominal
8,0%,Integer identifier corresponding to 21 distinc...,Admission source,Nominal
9,0%,Integer number of days between admission and d...,Time in hospital,Numeric


In [58]:
dict_concat = dict_concat.append(data_dictionary.iloc[27])

In [59]:
dict_concat

,% missing,Description and values,Feature name,Type
0,0%,Unique identifier of an encounter,Encounter ID,Numeric
1,0%,Unique identifier of a patient,Patient number,Numeric
2,2%,"Values: Caucasian, Asian, African American, Hi...",Race,Nominal
3,0%,"Values: male, female, and unknown/invalid",Gender,Nominal
4,0%,"Grouped in 10-year intervals: 0, 10), 10, 20),...",Age,Nominal
5,97%,Weight in pounds.,Weight,Numeric
6,0%,Integer identifier corresponding to 9 distinct...,Admission type,Nominal
7,0%,Integer identifier corresponding to 29 distinc...,Discharge disposition,Nominal
8,0%,Integer identifier corresponding to 21 distinc...,Admission source,Nominal
9,0%,Integer number of days between admission and d...,Time in hospital,Numeric


In [28]:
dict_concat.drop(dict_concat.index[26])

,% missing,Description and values,Feature name,Type
0,0%,Unique identifier of an encounter,Encounter ID,Numeric
1,0%,Unique identifier of a patient,Patient number,Numeric
2,2%,"Values: Caucasian, Asian, African American, Hi...",Race,Nominal
3,0%,"Values: male, female, and unknown/invalid",Gender,Nominal
4,0%,"Grouped in 10-year intervals: 0, 10), 10, 20),...",Age,Nominal
5,97%,Weight in pounds.,Weight,Numeric
6,0%,Integer identifier corresponding to 9 distinct...,Admission type,Nominal
7,0%,Integer identifier corresponding to 29 distinc...,Discharge disposition,Nominal
8,0%,Integer identifier corresponding to 21 distinc...,Admission source,Nominal
9,0%,Integer number of days between admission and d...,Time in hospital,Numeric


In [27]:
# set the type of the drug to 'nominal'
df_drug['Type'] = 'Nominal'

In [89]:
#insert the list into the dataframe replacing row 26 with the list of 23 drugs - use concat?
data_dict = pd.concat(data_dictionary, df_drug).reset_index(drop=True)

TypeError: first argument must be an iterable of pandas objects, you passed an object of type "DataFrame"